In [1]:
from time import sleep
from langame import LangameClient
from deep_translator import GoogleTranslator
from google.cloud.firestore import DocumentSnapshot
LANGUAGES = ["de", "es", "fr", "it", "ja", "ko", "pt", "ru", "zh-CN"]

c = LangameClient(path_to_config_file="../config.yaml")
i = 0
for e in c._firestore_client.collection("memes").stream():
    doc: DocumentSnapshot = e
    content = doc.to_dict()["content"]
    if not content:
        print(f"{doc.id} has no content")
        continue
    print("translating", doc.id)
    translated = {}
    for lang in LANGUAGES:
        translated[lang] = GoogleTranslator(source='en', target=lang).translate(content)
    print(f"Done translation of doc {doc.id}, n°{i}: {translated}")
    new_data = {"translated": translated}
    if "disabled" not in doc.to_dict():
        new_data["disabled"] = False
    if "confirmed" not in doc.to_dict():
        new_data["confirmed"] = False
    doc.reference.update(new_data)
    print("updated", doc.id, new_data)
    i += 1
    sleep(3pa)

translating 0FNnecFpEToBI1mwAIJF
Done translation of doc 0FNnecFpEToBI1mwAIJF, n°0: {'de': 'Haben Sie eine Behinderung oder eine chronische Krankheit?', 'es': '¿Tiene algún tipo de discapacidad o enfermedad crónica?', 'fr': 'Avez-vous un type de handicap ou de maladie chronique?', 'it': 'Hai qualche tipo di disabilità o malattia cronica?', 'ja': '何か障害や慢性疾患がありますか？', 'ko': '장애가 있거나 만성 질환이 있습니까?', 'pt': 'Você tem algum tipo de deficiência ou doença crônica?', 'ru': 'Есть ли у вас какой-либо тип инвалидности или хроническое заболевание?', 'zh-CN': '您有任何类型的残疾或慢性病吗？'}
updated 0FNnecFpEToBI1mwAIJF {'translated': {'de': 'Haben Sie eine Behinderung oder eine chronische Krankheit?', 'es': '¿Tiene algún tipo de discapacidad o enfermedad crónica?', 'fr': 'Avez-vous un type de handicap ou de maladie chronique?', 'it': 'Hai qualche tipo di disabilità o malattia cronica?', 'ja': '何か障害や慢性疾患がありますか？', 'ko': '장애가 있거나 만성 질환이 있습니까?', 'pt': 'Você tem algum tipo de deficiência ou doença crônica?', 'ru': 'Ест